In [ ]:
#| default_exp training

In [ ]:
#| export
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import tensorflow as tf
import os
import pandas as pd
from io import BytesIO

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#| export

class model_training():
  if not os.path.exists('models'):
        os.mkdir('models')
  def __init__(self):
    self.MODELS_DIR = 'models'
    self.MODEL_TF = self.MODELS_DIR + 'model'
    self.MODEL_NO_QUANT_TFLITE = self.MODELS_DIR + '/model_no_quant.tflite'
    self.MODEL_TFLITE = self.MODELS_DIR + '/model.tflite'
    self.MODEL_TFLITE_MICRO = self.MODELS_DIR + '/model.cc'
    self.data_dir = 'data/'

  def load_data(self, img_height, img_width, batch_size):
    """
    Loads data from the directory provided in data_dir
    """

    train_ds = tf.keras.utils.image_dataset_from_directory(
      self.data_dir,
      validation_split=0.2,
      subset="training",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)
    
    val_ds = tf.keras.utils.image_dataset_from_directory(
      self.data_dir,
      validation_split=0.2,
      subset="validation",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)


    return train_ds, val_ds


  #Train the model 180x180 @ 32 batch size
  def train_model(self, img_height, img_width, batch_size):
    """
    Trains the model
    """
    self.train_ds, self.val_ds = self.load_data(img_height, img_width, batch_size)

    class_names = self.train_ds.class_names
    
    #Enable caching for training
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = self.train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = self.val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    num_classes = len(class_names)

    model = Sequential([
      layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.20),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(num_classes)
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.summary()
    epochs=5
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )
    epochs_range = range(epochs)

    self.convert_model(model, train_ds)

    return model, history, epochs_range

  def convert_model(self, model, train_ds):
    """
    Convert the model into TFLite format
    """

    # Convert the model to the TensorFlow Lite format without quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    model_no_quant_tflite = converter.convert()

    # Save the model to disk
    open(self.MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

    # Convert the model with quantization.
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    #uncomment to enable end-to-end int8-model, i.e. input and output is also int8/uint8.

    #converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    #converter.inference_input_type = tf.int8
    #converter.inference_output_type = tf.int8

    def representative_dataset():
      for data in tf.data.Dataset.from_tensor_slices((train_ds)).batch(1).take(100):
        yield [data[0]]

    #converter.representative_dataset = representative_dataset
    tflite_model = converter.convert()

    # Save the model.
    with open(self.MODEL_TFLITE, 'wb') as f:
      f.write(tflite_model)

  def prediction(self, model):
    """Predicts on the image provided in the path.

    Args:
        model (tflite model): tflite model to be used in the prediction

    Returns:
        img: image predicted, result: formatted string for the result
    """

    path = '/data/1/1.png'
    img = cv2.imread(path)
    img = cv2.resize(img, (180,180))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    result = ("This image most likely belongs to {} with a {:.2f} percent confidence.".format(self.train_ds.class_names[np.argmax(score)], 100 * np.max(score)))
    
    return img, result

  def plot_statistics(self, history, epochs_range):
    """
    Plot the training statistics of the model to see over/underfitting etc.
    """

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    stats = plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    buff = BytesIO()
    stats.savefig(buff, format="png")
    
    return buff

    # Compare size table
  def plot_size(self,):

      # Calculate size
    size_no_quant_tflite = os.path.getsize(self.MODEL_NO_QUANT_TFLITE)
    size_tflite = os.path.getsize(self.MODEL_TFLITE)

    frame = pd.DataFrame.from_records(
        [["TensorFlow Lite", f"{size_no_quant_tflite} bytes "],
        ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
        columns = ["Model", "Size", ""], index="Model")
      
    return frame